## nfl lines machine learning project


source data for historical spreads and game scores
http://www.repole.com/sun4cast/data.html

In [61]:
import os

# define the root directory for the nfl code in $MLNLF_ROOT
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
dataRoot = "".join([codeDir, "data", os.path.sep])

os.chdir(codeDir)

print(codeDir)
print(dataRoot)

ERROR! Session/line number was not unique in database. History logging moved to new session 367
/Users/amit/repos/mlnfl/nfl/
/Users/amit/repos/mlnfl/nfl/data/


In [62]:
# warnings control
import warnings
# choose default, ignore, always
warnings.filterwarnings('default')


In [63]:
# import necessary modules
%matplotlib inline

from __future__ import division
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import madden

from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble

print ("pandas version ",pd.__version__)

pandas version  0.15.1


In [64]:
# location of lookup files
from referencedata import ReferenceData
lookupFiles = { 'teams' : {'file': 'nflTeams.csv' },
                'seasons' : {'file': 'seasons.csv' },
               }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

print ("lookupFiles = %s" % lookupFiles)
print ("lookupDir = %s" % lookupDir)

lookupFiles = {'seasons': {'file': 'seasons.csv'}, 'teams': {'file': 'nflTeams.csv'}}
lookupDir = /Users/amit/repos/mlnfl/nfl/data/lookup/


In [65]:
# import reference data
reference_data = ReferenceData(lookupDir)
reference_data.teams_df.head()

,city,mascot,league,division,year
team,,,,,
Baltimore Ravens,NaN,Ravens,afc,north,NaN
New England Patriots,NaN,Patriots,afc,east,NaN
Tennessee Titans,NaN,Titans,afc,south,NaN
Atlanta Falcons,NaN,Falcons,nfc,south,NaN
Tampa Bay Buccaneers,NaN,Buccaneers,nfc,south,NaN


In [66]:
###  multi-season training
reload(madden)

# testYears and trainYears need to be arrays
trainYears = [2013] #range(2008,2013) 
testYear = [2014]

# train on previous 3 yrs of data
testYear = [2015]
trainYears = range(testYear[0]-3,testYear[0]) 

# training data set - includes one extra year for prev yr record
seasons = np.array(trainYears)
print ("training seasons >> ", seasons)

training seasons >>  [2012 2013 2014]


In [69]:
reload(madden)

# get training data
# 1 - read all the games
path_to_lines = dataRoot + "lines/"
dfAllGames = madden.readGamesAll(path_to_lines, seasons)
# 2 - compile season record for all teams
dfAllTeams = madden.seasonRecord(dfAllGames, reference_data)
# 3 - apply season records and compute other fields for all games
dfAllGames = madden.processGames(dfAllGames, dfAllTeams, reference_data)
# 4 - remove extra year of data 
dfAllGames = dfAllGames[dfAllGames.season.isin(seasons)]

# use different test set 
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)
# 1 - read all the games
dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)  
# 2 - compile season record for all teams
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data) 
# 3 - apply season records and compute other fields for all games
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data) 
# 4 - remove extra year of data 
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]


results for >>  [2015]


In [76]:
# define independent variables for logistic regression
features = ['favoredRecord','underdogRecord',  # current year records of both teams
            'prevFavoredRecord','prevUnderdogRecord', # prev year records, helps early in season when only few games played
            'gameWeek',  # week in season, should make a good/bad record later in season more important
            'absLine',  # absolute value of spread since favored team already determined
            'divisionGame', # T/F, usually more competitive rivalry games, i.e. bad teams still win home division games.
            'favoredHomeGame', # T/F, important since output of classifier is "did the favored team win?"
            ]

# run the classifer
classifier = svm.SVC(kernel='poly',probability=True)
#classifier = linear_model.LogisticRegression(C=1e5)
#classifier = ensemble.RandomForestClassifier(n_estimators=10)
mlClassifier = madden.runScikitClassifier(dfAllGames,features,classifier)

# apply results of logistic regression to the test set
#dfPredict = madden.predictGames(dfGamesTest,mlClassifier,features)

# apply ranking logic and determine scoring outcomes for league
#dfAll = madden.rankGames(dfPredict,reference_data,seasonTest[0])

training data accuracy =  0.71484375


In [77]:
# diagnostics ... weekly outcomes
g = dfAll.groupby('gameWeek')['lineScore','probaScore1','probaScore2','probaScore3'].sum()
print(g)

g.sum()

          lineScore  probaScore1  probaScore2  probaScore3
gameWeek                                                  
1               NaN          NaN            0            0


lineScore     NaN
probaScore1   NaN
probaScore2     0
probaScore3     0
dtype: float64

In [78]:
# predict one week of current season
iweek = 1
reload(madden)

# use different test set - current year
testYear = [2015]
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)

dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]

# pick only this weeks games for predict
dfTest = dfGamesTest[dfGamesTest.gameWeek == iweek]

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfTest,mlClassifier,features)
# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict, reference_data, seasonTest[0])



results for >>  [2015]


In [82]:
# display weekly ranking output

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

dispCols = ['season','gameWeek','Visitor','visitorRecord','Home Team','homeRecord',
            'Line','prevFavoredRecord','prevUnderdogRecord','predict_proba',
            'lineGuess','probaGuess', 'probaAbsGuess', 'predictTeam']


dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
guessCol = 'probaGuess'
predictCols = ['predictTeam', 'predict_proba', guessCol, 'favorite','lineGuess', 'Line']
dfAll[predictCols].sort(guessCol, ascending=False)

,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
8699,Dallas Cowboys,0.696270,16,Dallas Cowboys,15,5.5
8698,Denver Broncos,0.693593,15,Denver Broncos,14,4.5
8686,New England Patriots,0.685329,14,New England Patriots,7,3.0
8694,Arizona Cardinals,0.679768,13,Arizona Cardinals,3,2.5
8696,Tampa Bay Buccaneers,0.677770,12,Tampa Bay Buccaneers,9,3.0
8689,New York Jets,0.676639,11,New York Jets,8,3.0
8695,San Diego Chargers,0.674916,10,San Diego Chargers,4,2.5
8688,Houston Texans,0.664795,9,Houston Texans,1,1.5
8687,Green Bay Packers,0.662605,8,Green Bay Packers,16,-6.5
8693,Seattle Seahawks,0.648897,7,Seattle Seahawks,13,-3.5


In [84]:
dfAll

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine,...,predict_proba_abs,predictWin,lineGuess,probaGuess,probaAbsGuess,lineScore,probaScore1,probaScore2,probaScore3,predictTeam
8688,9/13/15,Kansas City Chiefs,NaN,Houston Texans,NaN,1.5,NaN,2015,NaN,NaN,...,0.164795,0,1,9,9,NaN,NaN,0,0,Houston Texans
8701,9/14/15,Minnesota Vikings,NaN,San Francisco 49ers,NaN,-2.0,NaN,2015,NaN,NaN,...,0.147804,0,2,6,6,NaN,NaN,0,0,Minnesota Vikings
8694,9/13/15,New Orleans Saints,NaN,Arizona Cardinals,NaN,2.5,NaN,2015,NaN,NaN,...,0.179768,0,3,13,13,NaN,NaN,0,0,Arizona Cardinals
8695,9/13/15,Detroit Lions,NaN,San Diego Chargers,NaN,2.5,NaN,2015,NaN,NaN,...,0.174916,0,4,10,10,NaN,NaN,0,0,San Diego Chargers
8700,9/14/15,Philadelphia Eagles,NaN,Atlanta Falcons,NaN,-3.0,NaN,2015,NaN,NaN,...,0.146854,0,5,5,5,NaN,NaN,0,0,Philadelphia Eagles
8690,9/13/15,Indianapolis Colts,NaN,Buffalo Bills,NaN,-3.0,NaN,2015,NaN,NaN,...,0.145155,0,6,2,2,NaN,NaN,0,0,Indianapolis Colts
8686,9/10/15,Pittsburgh Steelers,NaN,New England Patriots,NaN,3.0,NaN,2015,NaN,NaN,...,0.185329,0,7,14,14,NaN,NaN,0,0,New England Patriots
8689,9/13/15,Cleveland Browns,NaN,New York Jets,NaN,3.0,NaN,2015,NaN,NaN,...,0.176639,0,8,11,11,NaN,NaN,0,0,New York Jets
8696,9/13/15,Tennessee Titans,NaN,Tampa Bay Buccaneers,NaN,3.0,NaN,2015,NaN,NaN,...,0.177770,0,9,12,12,NaN,NaN,0,0,Tampa Bay Buccaneers
8692,9/13/15,Carolina Panthers,NaN,Jacksonville Jaguars,NaN,-3.5,NaN,2015,NaN,NaN,...,0.145887,0,10,4,4,NaN,NaN,0,0,Carolina Panthers


In [80]:
# display weekly ranking output for spread method

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

predictCols = ['favorite','lineGuess', 'absLine','Line', 'favoredHomeGame', 'divisionGame', 'favoredRecord']

sortCols = ['absLine','favoredHomeGame', 'divisionGame', 'favoredRecord', 'favorite']
dfSpread = dfAll[predictCols].sort(sortCols , ascending=False)
#print(dfSpread.to_csv(sys.stdout,sep=',', index=False))
dfSpread

,favorite,lineGuess,absLine,Line,favoredHomeGame,divisionGame,favoredRecord
8687,Green Bay Packers,16,6.5,-6.5,0,1,0
8699,Dallas Cowboys,15,5.5,5.5,1,1,0
8698,Denver Broncos,14,4.5,4.5,1,0,0
8693,Seattle Seahawks,13,3.5,-3.5,0,1,0
8691,Miami Dolphins,12,3.5,-3.5,0,0,0
8697,Cincinnati Bengals,11,3.5,-3.5,0,0,0
8692,Carolina Panthers,10,3.5,-3.5,0,0,0
8696,Tampa Bay Buccaneers,9,3.0,3.0,1,0,0
8689,New York Jets,8,3.0,3.0,1,0,0
8686,New England Patriots,7,3.0,3.0,1,0,0


In [83]:
# display details for a single week
dispWeek = 1
dispCols = ['season','gameWeek','Visitor','Home Team',
            'visitorRecord', 'homeRecord', 'favoredRecord','underdogRecord',
            'prevFavoredRecord','Line','predict_proba','Visitor Score',
            'Home Score','favoredWin','predictWin','lineGuess','probaGuess', 'lineScore','probaScore1','probaScore2']

print(dfAll[dfAll.gameWeek == dispWeek]['lineScore'].sum())
dfAll[dfAll.gameWeek == dispWeek][dispCols]

nan


,season,gameWeek,Visitor,Home Team,visitorRecord,homeRecord,favoredRecord,underdogRecord,prevFavoredRecord,Line,predict_proba,Visitor Score,Home Score,favoredWin,predictWin,lineGuess,probaGuess,lineScore,probaScore1,probaScore2
8688,2015,1,Kansas City Chiefs,Houston Texans,0,0,0,0,0.5625,1.5,0.664795,NaN,NaN,NaN,0,1,9,NaN,NaN,0
8701,2015,1,Minnesota Vikings,San Francisco 49ers,0,0,0,0,0.4375,-2.0,0.647804,NaN,NaN,NaN,0,2,6,NaN,NaN,0
8694,2015,1,New Orleans Saints,Arizona Cardinals,0,0,0,0,0.6875,2.5,0.679768,NaN,NaN,NaN,0,3,13,NaN,NaN,0
8695,2015,1,Detroit Lions,San Diego Chargers,0,0,0,0,0.5625,2.5,0.674916,NaN,NaN,NaN,0,4,10,NaN,NaN,0
8700,2015,1,Philadelphia Eagles,Atlanta Falcons,0,0,0,0,0.6250,-3.0,0.646854,NaN,NaN,NaN,0,5,5,NaN,NaN,0
8690,2015,1,Indianapolis Colts,Buffalo Bills,0,0,0,0,0.6875,-3.0,0.645155,NaN,NaN,NaN,0,6,2,NaN,NaN,0
8686,2015,1,Pittsburgh Steelers,New England Patriots,0,0,0,0,0.7500,3.0,0.685329,NaN,NaN,NaN,0,7,14,NaN,NaN,0
8689,2015,1,Cleveland Browns,New York Jets,0,0,0,0,0.2500,3.0,0.676639,NaN,NaN,NaN,0,8,11,NaN,NaN,0
8696,2015,1,Tennessee Titans,Tampa Bay Buccaneers,0,0,0,0,0.1250,3.0,0.677770,NaN,NaN,NaN,0,9,12,NaN,NaN,0
8692,2015,1,Carolina Panthers,Jacksonville Jaguars,0,0,0,0,0.4375,-3.5,0.645887,NaN,NaN,NaN,0,10,4,NaN,NaN,0


In [17]:
dfAll.gameWeek.head()

8674    17
8678    17
8670    17
8679    17
8680    17
Name: gameWeek, dtype: int64

In [ ]:
# diagnostics ... plot outcomes of games
nWin = dfAllGames.favoredWin.index.tolist()
nLose = dfAllGames[dfAllGames['favoredWin'] == 0].index.tolist()

plt.plot(dfAllGames.favoredRecord[nWin],dfAllGames.underdogRecord[nWin],'bx')
plt.plot(dfAllGames.favoredRecord[nLose],dfAllGames.underdogRecord[nLose],'rx')

In [ ]:
# diagnostics ... logreg variables

# print classifiers
print("classifiers > ",logreg.classes_)

# coefficients
cc = logreg.coef_
cc.shape
print("coeffs")
for ii,ff in enumerate(features):
    print(ii,ff,cc[0][ii])
    

In [ ]:
# run one year of rolling training, control width of training window w/ trainFreq
import runMadden
reload(runMadden)
reload(madden)

# need to be arrays
trainYears = [2012] #range(2008.2013) 
testYear = [2013]
trainFreq = 6

dd = runMadden.runSeasonRolling(trainYears,testYear,olookups,trainFreq)
dd

In [32]:
# loop over years to test trainFreq
reload(runMadden)
reload(madden)

ddd = None
for y in range(2008,2014):
    trainYears = [y]
    testYear = [y+1]
    trainFreq = 20
    dd = runMadden.runSeasonRolling(trainYears, testYear, reference_data, path_to_lines, trainFreq)
    if ddd is None:
        ddd = dd
    else:
        ddd = ddd.append(dd)
        
ddd        

season 2009, week 1
training data accuracy =  0.7421875
season 2009, week 2
training data accuracy =  0.7421875
season 2009, week 3
training data accuracy =  0.7421875
season 2009, week 4
training data accuracy =  0.7421875
season 2009, week 5
training data accuracy =  0.7421875
season 2009, week 6
training data accuracy =  0.7421875
season 2009, week 7
training data accuracy =  0.7421875
season 2009, week 8
training data accuracy =  0.7421875
season 2009, week 9
training data accuracy =  0.7421875
season 2009, week 10
training data accuracy =  0.7421875
season 2009, week 11
training data accuracy =  0.7421875
season 2009, week 12
training data accuracy =  0.7421875
season 2009, week 13
training data accuracy =  0.7421875
season 2009, week 14
training data accuracy =  0.7421875
season 2009, week 15
training data accuracy =  0.7421875
season 2009, week 16
training data accuracy =  0.7421875
season 2009, week 17
training data accuracy =  0.7421875
season 2010, week 1
training data accura

KeyboardInterrupt: 

In [ ]:
# diagnostics ... plot outcomes of games
nWin = dfAll.favoredWin.index.tolist()
nLose = dfAll[dfAll['favoredWin'] == 0].index.tolist()

plt.plot(dfAll.favoredRecord[nWin],dfAll.underdogRecord[nWin],'bx')
plt.plot(dfAll.favoredRecord[nLose],dfAll.underdogRecord[nLose],'rx')

# pct correct according to spread
print (len(nWin)/ (len(nWin) + len(nLose)))

# pct correct according to model
nWin = dfAll.predictWin.index.tolist()
nLose = dfAll[dfAll['predictWin'] == 0].index.tolist()
print (len(nWin)/ (len(nWin) + len(nLose)))


### loop over multiple years
see how different classifiers or test/train combos perform over the years


In [33]:
import runMadden
import walsh
import madden

path_to_lines = dataRoot + "lines/"

In [43]:
# loop over multiple years

import runMadden
import walsh
reload(madden)
reload(runMadden)


trainStart = 2008
trainLen = 3

mlClassifier = svm.SVC(kernel='poly',probability=True)
dfSVM = runMadden.runSeasonLoop(trainStart, trainLen, mlClassifier, path_to_lines, reference_data)

mlClassifier = linear_model.LogisticRegression(C=1e5)
dfLog = runMadden.runSeasonLoop(trainStart, trainLen, mlClassifier, path_to_lines, reference_data)
#dfLoop = dfSVM


2011 [2008, 2009, 2010]
training data accuracy =  0.70703125
1670 <type 'int'>
2012 [2009, 2010, 2011]
training data accuracy =  0.716145833333
1632 <type 'int'>
2013 [2010, 2011, 2012]
training data accuracy =  0.71484375
1653 <type 'int'>
2014 [2011, 2012, 2013]
training data accuracy =  0.716145833333
1669.0 <type 'numpy.float64'>
2011 [2008, 2009, 2010]
training data accuracy =  0.697916666667
1670 <type 'int'>
2012 [2009, 2010, 2011]
training data accuracy =  0.712239583333
1632 <type 'int'>
2013 [2010, 2011, 2012]
training data accuracy =  0.690104166667
1653 <type 'int'>
2014 [2011, 2012, 2013]
training data accuracy =  0.6953125
1669.0 <type 'numpy.float64'>


In [44]:
dfLog

,lineScore,probaScore1,probaScore2,probaScore3,trainYears,classifierType,classifier
season,,,,,,,
2011,34,49,42,33,[2008 2009 2010],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."
2012,-14,-2,20,2,[2009 2010 2011],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."
2013,2,28,40,35,[2010 2011 2012],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."
2014,0,0,-40,-52,[2011 2012 2013],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."


In [45]:
dfSVM

,lineScore,probaScore1,probaScore2,probaScore3,trainYears,classifierType,classifier
season,,,,,,,
2011,34,22,33,40,[2008 2009 2010],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."
2012,-14,21,46,40,[2009 2010 2011],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."
2013,2,8,-11,-33,[2010 2011 2012],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."
2014,0,81,92,109,[2011 2012 2013],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."


In [39]:
trainYears = [2013] 
testYear = [2014]

seasons = np.array(trainYears)
seasonTest = np.array(testYear)

# test new wrapper object
oSeason = walsh.SeasonClassifier(path_to_lines, reference_data)
oSeason.readData(seasons, 'train')
oSeason.readData(seasonTest, 'test')
oSeason.features = features
mlClassifier = linear_model.LogisticRegression(C=1e5)
oSeason.classifier = mlClassifier
oSeason.runClassifier('favoredWin')
oSeason.predict()
oSeason.predictAccuracy('train')
oSeason.rank()
oSeason.predictSummary(week=12, guessCol='probaGuess')

training data accuracy =  0.70703125
('train predict accuracy = ', 0.70703125)


walsh.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])


,predictTeam,predict_proba,probaGuess,favorite,Line
8596,Indianapolis Colts,0.923455,16,Indianapolis Colts,14.0
8599,Philadelphia Eagles,0.891147,15,Philadelphia Eagles,11.0
8593,Buffalo Bills,0.797302,14,Buffalo Bills,4.5
8603,San Francisco 49ers,0.795938,13,San Francisco 49ers,8.5
8597,Green Bay Packers,0.778764,12,Green Bay Packers,-9.5
8591,Kansas City Chiefs,0.771934,11,Kansas City Chiefs,-7.0
8602,Denver Broncos,0.767666,10,Denver Broncos,7.0
8598,New England Patriots,0.764685,9,New England Patriots,7.0
8600,San Diego Chargers,0.737630,8,San Diego Chargers,5.0
8601,Seattle Seahawks,0.708936,7,Seattle Seahawks,6.5


In [36]:
oSeason.predictSummary(week=11, guessCol='probaGuess')

,predictTeam,predict_proba,probaGuess,favorite,Line
8587,San Diego Chargers,0.896355,16,San Diego Chargers,10.0
8585,Denver Broncos,0.736736,15,Denver Broncos,-9.5
8577,Miami Dolphins,0.724121,14,Miami Dolphins,5.0
8581,Green Bay Packers,0.696934,13,Green Bay Packers,5.5
8588,Arizona Cardinals,0.675204,12,Arizona Cardinals,2.0
8582,Kansas City Chiefs,0.675102,11,Kansas City Chiefs,1.5
8590,Pittsburgh Steelers,0.671690,10,Pittsburgh Steelers,-5.5
8586,Washington Redskins,0.666699,9,Washington Redskins,7.0
8583,New Orleans Saints,0.651453,8,New Orleans Saints,7.0
8579,Chicago Bears,0.619544,7,Chicago Bears,3.0


In [35]:
oSeason.predictSummary(week=12, guessCol='probaGuess')

,predictTeam,predict_proba,probaGuess,favorite,Line
8596,Indianapolis Colts,0.910224,16,Indianapolis Colts,14.0
8599,Philadelphia Eagles,0.873100,15,Philadelphia Eagles,11.0
8591,Kansas City Chiefs,0.795660,14,Kansas City Chiefs,-7.0
8593,Buffalo Bills,0.767750,13,Buffalo Bills,4.5
8603,San Francisco 49ers,0.766246,12,San Francisco 49ers,8.5
8597,Green Bay Packers,0.747365,11,Green Bay Packers,-9.5
8602,Denver Broncos,0.735228,10,Denver Broncos,7.0
8598,New England Patriots,0.731976,9,New England Patriots,7.0
8600,San Diego Chargers,0.702623,8,San Diego Chargers,5.0
8601,Seattle Seahawks,0.671804,7,Seattle Seahawks,6.5


In [34]:
for i, feature in enumerate(features):
    print("Coeficient {0} = {1}".format(feature, oSeason.classifier.coef_[0, i]))

Coeficient favoredRecord = 1.91252475964
Coeficient underdogRecord = -1.54131295184
Coeficient prevFavoredRecord = 0.0754170412348
Coeficient prevUnderdogRecord = 0.730741963582
Coeficient gameWeek = -0.0126558814763
Coeficient absLine = 0.101668474925
Coeficient divisionGame = 0.248762869133
Coeficient favoredHomeGame = 0.559291599483


In [31]:
oSeason.classifier.coef_?

In [ ]:
reload(madden)
# loop over all train sets and read data

# create a collection of season objects to test various classifiers and parameters
path_to_lines = dataRoot + "lines/"
trainStart = 2012
trainLen = 1
testYear = 0

features = ['favoredRecord','underdogRecord',  # current year records of both teams
            'prevFavoredRecord','prevUnderdogRecord', # prev year records, helps early in season when only few games played
            'gameWeek',  # week in season, should make a good/bad record later in season more important
            'absLine',  # absolute value of spread since favored team already determined
            'divisionGame', # T/F, usually more competitive rivalry games, i.e. bad teams still win home division games.
            'favoredHomeGame', # T/F, important since output of classifier is "did the favored team win?"
            ]

oCollection = walsh.SeasonClassifierCollection()
mlClassifier = svm.SVC(kernel='poly',probability=True)

while testYear < 2014:
    # determine test and train years
    testYear = trainStart + trainLen
    trainYears = range(testYear-trainLen,testYear)


    print (trainYears, testYear)

    # define test and train years
    seasons = np.array(trainYears)
    seasonTest = np.array([testYear]) # should be only one year
    
    #mlClassifier = svm.SVC(kernel='poly',probability=True)
    mlClassifier = linear_model.LogisticRegression(C=1e5)
    classifier_str = str(mlClassifier)[:11]
    description_str = "%s-train-%s-test-%s" % (classifier_str, str(trainYears), str(testYear))
    
    print(description_str)
    
    # create SeasonClassifier object
    oSeason = walsh.SeasonClassifier(path_to_lines, reference_data)
    oSeason.readData(seasons, 'train')
    oSeason.readData(seasonTest, 'test')
    oSeason.features = features

    oSeason.classifier = mlClassifier
    oSeason.runClassifier('favoredWin')
    oSeason.predict()
    oSeason.predictAccuracy('test')
    oSeason.rank()

    oCollection.addSeasonClassifier(oSeason, description_str)
    
    trainStart += 1

In [ ]:
oss = oCollection.collection[description_str]
oss.predictAccuracy('test')

In [ ]:
# copy from one collection to another

reload(madden)
newCollection = madden.SeasonClassifierCollection()

for k,v in oCollection.collection.items():
    print (k,v)
    newCollection.addSeasonClassifier(v,k)
    
    

In [ ]:
oCollection.summary()

In [ ]:
oCollection.listCollection()

In [ ]:
# copy from one collection to another and change classifier and compute

reload(madden)
svm_Collection = madden.SeasonClassifierCollection()

for k,v in oCollection.collection.items():
    mlClassifier = svm.SVC(kernel='poly',probability=True)
    print (k,v)
    
    v.setClassifier(mlClassifier)
    v.runClassifier('favoredWin')
    v.predict()
    v.predictAccuracy('test')
    v.rank()
    newCollection.addSeasonClassifier(v,k)

newCollection.summary()